In [1]:
from ultralytics import YOLO
# Burada kullanacağımız modeli seçiyoruz.
model= YOLO("yolov8l.pt") 

In [2]:
import cv2
import numpy as np

In [3]:
# Tracking işlemi yapıldığı için RAM dolup kod kapanmasın diye bunu çalıştırıyoruz.
# Bazı donanım ya da kütüphane sürümlerine gerekli olmayabilir.

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [5]:
kamera= cv2.VideoCapture('video3.mp4')
font = cv2.FONT_HERSHEY_SIMPLEX


# burada her bir şerit için bir bölge belirliyoruz.
# Bu videoda 3 tane şerit olduğu için 3 bölge belirledim
region1=np.array([(180,0),(320,0),(210,360),(-70,360)])
region1 = region1.reshape((-1,1,2))


region2=np.array([(320,0),(405,0),(470,360),(210,360)])
region2 = region2.reshape((-1,1,2))


region3=np.array([(405,0),(500,0),(690,360),(470,360)])
region3 = region3.reshape((-1,1,2))


# Her bir bölgedeki araçların id'sini tutacak değişkenler belirliyoruz.
# Her bir id'nin sadece bir kere gözükmesi için küme tipinde belirledim

sol_bolge=set()
orta_bolge=set()
sag_bolge=set()






while True:
    
 

    ret, frame = kamera.read()

    if not ret:
        break
        
    # Görseli RGB formatına çevirip modele veriyoruz.
    rgb_img=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results = model.track(rgb_img, persist=True, verbose=False)
    labels=results[0].names
 

   
    
    for i in range(len(results[0].boxes)):
        x1,y1,x2,y2=results[0].boxes.xyxy[i]
        score=results[0].boxes.conf[i]
        cls=results[0].boxes.cls[i]
        ids=results[0].boxes.id[i]
        x1,y1,x2,y2,score,cls,ids=int(x1),int(y1),int(x2),int(y2),float(score),int(cls),int(ids)
        
        name=labels[cls]

        
       
        if score<0.1:
            continue
        # Eğer nesne aşağıdaki araçlardan birisi değilse ekranda değerlendirmeye alınmıyor
        if name!='car' and name!='truck'  and name!='motorcycle' and name!='bicycle' :
            continue
        
        # Nesnelerin orta noktasını buluyoruz.
        cx=int(x1/2+x2/2)
        cy=int(y1/2+y2/2)
        
        # Nesnenin orta noktası 1. bölgenin içinde mi onu kontrol ediyoruz.
        # Eğer içindeyse bu değer 1 olacak
        inside_region1=cv2.pointPolygonTest(region1,(cx,cy),False)
        if inside_region1>0:
            
            # Nesnenin id'yi sol bölge için olan kümenin içine ekliyoruz.
            sol_bolge.add(ids)
            
            # Sonra bu nesnenin id'si diğer kümelerde de var mı diye bakıyoruz.
            # Eğer içindeyse o zaman bu araç başka bir şeritte daha olmuştur.
            # Bu durumda şerit değiştirmiş demektir.
            if ids in orta_bolge or ids in sag_bolge:
                
                # Şerit değiştirmiş aracı ekranda gösteriyoruz.
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            
            

        # Aynısını 2. bölge için yapıyoruz
        inside_region2=cv2.pointPolygonTest(region2,(cx,cy),False)
        if inside_region2>0:
            orta_bolge.add(ids)
            if ids in sol_bolge or ids in sag_bolge:
                
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            
        # Aynısını 3. bölge için yapıyoruz
        inside_region3=cv2.pointPolygonTest(region3,(cx,cy),False)
        if inside_region3>0:
            sag_bolge.add(ids)   
            if ids in orta_bolge or ids in sol_bolge:
                
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
        

        
  
    # İsterseniz bölgeleri ekranda gösterebilirsiniz.
    # cv2.polylines(frame,[region1],True,(255,0,0),3)
    # cv2.polylines(frame,[region2],True,(0,0,255),3)
    # cv2.polylines(frame,[region3],True,(0,255,0),3)
    
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
kamera.release()
cv2.destroyAllWindows()